In [1]:
import numpy as np
import matplotlib.pyplot as plt
from torch import nn
from torchvision import models
import torch
import gym, sys
import cv2
from tqdm import tqdm
from IPython.display import clear_output
from array2gif import write_gif
def data_transform(frame) :
    # breakout-v4
    # [width, height, channels] (기존)
    # [channels, height, width] (변경)
    frame = np.array(frame).transpose(2,1,0)
    frame = np.expand_dims(frame, 0) 
    assert np.sum(np.isnan(frame)) == 0, "state error!"
    return frame.astype(np.float32)
    
from rlxai.gym_wrapper import FireResetEnv , WarpFrame , ScaledFloatFrame , FrameStack


/opt/conda/envs/xai/lib/python3.8/site-packages/ale_py/roms/__init__.py:94: DeprecationWarning: Automatic importing of atari-py roms won't be supported in future releases of ale-py. Please migrate over to using `ale-import-roms` OR an ALE-supported ROM package. To make this warning disappear you can run `ale-import-roms --import-from-pkg atari_py.atari_roms`.For more information see: https://github.com/mgbellemare/Arcade-Learning-Environment#rom-management
  _RESOLVED_ROMS = _resolve_roms()


In [2]:
env = gym.make("Breakout-v4") # Deterministic
if 'FIRE' in env.unwrapped.get_action_meanings():
    env = FireResetEnv(env)
env = WarpFrame(env)
env = ScaledFloatFrame(env)
env = FrameStack(env,k=3)

random_seed = 215161  # set random seed if required (0 = no random seed)
action_dim = env.action_space.n
device = "cuda:1"
from rlxai.agent.agent_selection import get_agent
model_name = "PER"
save_path = "./../model"
model,_,_ = get_agent(model_name=model_name,save_path=save_path,device=device,action_dim=action_dim,state_dim=3,load_model=True)

A.L.E: Arcade Learning Environment (version +978d2ce)
[Powered by Stella]


load
...Load model from ../model/PER/model.pt...


In [3]:
if random_seed:
    print("--------------------------------------------------------------------------------------------")
    print("setting random seed to ", random_seed)
    torch.manual_seed(random_seed)
    env.seed(random_seed)
    np.random.seed(random_seed)

--------------------------------------------------------------------------------------------
setting random seed to  215161


In [4]:
ACTION_MEANING = env.unwrapped.get_action_meanings()

In [5]:
env.render(mode="rgb_array").shape # 

(210, 160, 3)

In [11]:
s = env.reset()
img_collection = [env.render(mode="rgb_array").transpose(1,0,2)]
iter = 0
ACTION_LIST = []
R = 0
info = {'lives' : 5}
last_lives = info['lives']
while True:
    action_dict = model.select_action(data_transform(s),False)
    if info["lives"] < last_lives :
        action = 1
        last_lives = info["lives"]
    else :
        action = int(action_dict["action"])
    s, r, done, info = env.step(action)
    R += r
    ACTION_LIST.append(ACTION_MEANING[action])
    #############################################
    img_collection.append(env.render(mode="rgb_array").transpose(1,0,2))
    iter += 1
    if done:
        break
# print(", ".join(ACTION_LIST))
print(f"Reward : {R}")
if R < 60 :
    print(f"Reward : {R} is very small so we don't make gif file")
else :
    file_path = "./breakout.gif"
    write_gif(img_collection, file_path)
    print(f"Reward : {R} save gif...")

Reward : 36.0
Reward : 36.0 is very small so we don't make gif file


In [15]:
# write_gif([img for img in img_collection[0:20]], file_path)